In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from time import time
from sklearn.metrics import log_loss, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import grid_search as gs
import sklearn.cross_validation as cv

#grouped = pd.read_csv("grouped.csv")
file_Name = "grouped_pickle"
fileObject = open(file_Name,'r')
grouped = pickle.load(fileObject)







In [2]:
from sklearn.cross_validation import train_test_split

#Adjust name of some columns to be more intuitive
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html
#df.rename(index=str, columns={"A": "a", "B": "c"})
grouped.rename(index=str, columns={"ScanCount": "TotalUnitsSold", "Upc": "UniqueUnitsSold"})

#for some reason after a dataframe is grouped the column order change.
#Rearrange them in a way that will help with modeling.
#Get a list of the exact column names, so they can be arranged.

#https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
#df = df[['mean', 4,3,2,1]]
grouped = grouped[[
'Weekday',
'Weekpart',
'ScanCount',
'ReturnedItem',
'Upc',
'1-HR PHOTO',
'ACCESSORIES',
'AUTOMOTIVE',
'BAKERY',
'BATH AND SHOWER',
'BEAUTY',
'BEDDING',
'BOOKS AND MAGAZINES',
'BOYS WEAR',
'BRAS & SHAPEWEAR',
'CAMERAS AND SUPPLIES',
'CANDY, TOBACCO, COOKIES',
'CELEBRATION',
'COMM BREAD',
'CONCEPT STORES',
'COOK AND DINE',
'DAIRY',
'DSD GROCERY',
'ELECTRONICS',
'FABRICS AND CRAFTS',
'FINANCIAL SERVICES',
'FROZEN FOODS',
'FURNITURE',
'GIRLS WEAR, 4-6X  AND 7-14',
'GROCERY DRY GOODS',
'HARDWARE',
'HEALTH AND BEAUTY AIDS',
'HOME DECOR',
'HOME MANAGEMENT',
'HORTICULTURE AND ACCESS',
'HOUSEHOLD CHEMICALS/SUPP',
'HOUSEHOLD PAPER GOODS',
'IMPULSE MERCHANDISE',
'INFANT APPAREL',
'INFANT CONSUMABLE HARDLINES',
'JEWELRY AND SUNGLASSES',
'LADIES SOCKS',
'LADIESWEAR',
'LARGE HOUSEHOLD GOODS',
'LAWN AND GARDEN',
'LIQUOR,WINE,BEER',
'MEAT - FRESH & FROZEN',
'MEDIA AND GAMING',
'MENS WEAR',
'MENSWEAR',
'OFFICE SUPPLIES',
'OPTICAL - FRAMES',
'OPTICAL - LENSES',
'OTHER DEPARTMENTS',
'PAINT AND ACCESSORIES',
'PERSONAL CARE',
'PETS AND SUPPLIES',
'PHARMACY OTC',
'PHARMACY RX',
'PLAYERS AND ELECTRONICS',
'PLUS AND MATERNITY',
'PRE PACKED DELI',
'PRODUCE',
'SEAFOOD',
'SEASONAL',
'SERVICE DELI',
'SHEER HOSIERY',
'SHOES',
'SLEEPWEAR/FOUNDATIONS',
'SPORTING GOODS',
'SWIMWEAR/OUTERWEAR',
'TOYS',
'WIRELESS',
'TripType']]



#Now that the data has been prepared get ready to evaluate models!
#Example code from project 2 - Student Intervention
# Extract feature columns
feature_cols = list(grouped.columns[:-1])

# Extract target column 'passed'
target_col = 'TripType'


# Show the list of columns
#print "Feature columns:\n{}".format(feature_cols)
#print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
#X_all = grouped[feature_cols]
#y_all = grouped[target_col]

#https://stackoverflow.com/questions/19392226/attributeerror-dataframe-object-has-no-attribute
train=grouped.sample(frac=0.5,random_state=61)
test=grouped.drop(train.index)

#Classification algorithms needs the same classes present in both the train and test datasets, as well as each fold in k-fold cross-validation.
#sklearn does not have a builtin method to increase the sampling of low frequency classes, so I will do it manually.
#TripType 14 only shows up 4 times in the entire dataset.  So I will increase copies of it in the train dataset.
#This way algorithms won't error out.  It also shows up 2 times in the test dataset.


train_14 = train[train['TripType']== 14]
train = pd.concat([train, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14
                    , train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14
                    , train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14
                    , train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14
                    , train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14
                    , train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14, train_14])


# Extract feature columns
feature_cols = list(train.columns[:-1])

# Extract target column 'passed'
target_col = 'TripType'


X_train = train[feature_cols]
y_train = train[target_col]

X_test = test[feature_cols]
y_test = test[target_col]





# TODO: Split the data into training and testing sets using the given feature as the target
#X_train, X_test, y_train, y_test = (None, None, None, None)
#X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=.5, random_state=61)

# Show the results of the split
#print "Training set has {} samples.".format(X_train.shape[0])
#print "Testing set has {} samples.".format(X_test.shape[0])

#print y_test
#print X_train

In [6]:

#train['TripType'].value_counts()
len(train['TripType'].unique())
#print train
#print train_14
#train_14.count()
#print train_14

#type(train_2)

38

In [5]:
#test['TripType'].value_counts()
len(test['TripType'].unique())


38

In [7]:
# Random Forest
rfc = RandomForestClassifier(n_jobs=1,oob_score = True,random_state=345) 

param_grid = { 
    'n_estimators': [200, 500, 1000, 1500, 2000, 2500],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'class_weight': ['balanced', None]
}

CV_rfc = gs.GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3, scoring='log_loss', pre_dispatch='n_jobs/2')
CV_rfc.fit(X_train, y_train)
print CV_rfc.best_params_

#{'max_features': 'log2', 'n_estimators': 2500, 'class_weight': None}

#http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
for params, mean_score, scores in CV_rfc.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))





{'max_features': 'log2', 'n_estimators': 2500, 'class_weight': None}
-1.525 (+/-0.118) for {'max_features': 'auto', 'n_estimators': 200, 'class_weight': 'balanced'}
-1.402 (+/-0.118) for {'max_features': 'auto', 'n_estimators': 500, 'class_weight': 'balanced'}
-1.345 (+/-0.089) for {'max_features': 'auto', 'n_estimators': 1000, 'class_weight': 'balanced'}
-1.314 (+/-0.087) for {'max_features': 'auto', 'n_estimators': 1500, 'class_weight': 'balanced'}
-1.302 (+/-0.088) for {'max_features': 'auto', 'n_estimators': 2000, 'class_weight': 'balanced'}
-1.293 (+/-0.088) for {'max_features': 'auto', 'n_estimators': 2500, 'class_weight': 'balanced'}
-1.525 (+/-0.118) for {'max_features': 'sqrt', 'n_estimators': 200, 'class_weight': 'balanced'}
-1.402 (+/-0.118) for {'max_features': 'sqrt', 'n_estimators': 500, 'class_weight': 'balanced'}
-1.345 (+/-0.089) for {'max_features': 'sqrt', 'n_estimators': 1000, 'class_weight': 'balanced'}
-1.314 (+/-0.087) for {'max_features': 'sqrt', 'n_estimators':

In [3]:
clf = RandomForestClassifier(n_jobs=1,max_features= 'log2' ,n_estimators=1500, class_weight=None, oob_score = True, random_state=345)
clf.fit(X_train, y_train) 
y_pred=clf.predict_proba(X_test)
y_true=y_test
y_true=y_true.values
classifier_results=log_loss(y_true, y_pred)
print classifier_results

#1.24542628505


1.24542628505


In [4]:
# Ada Boost

rfc = AdaBoostClassifier(random_state=345) 

param_grid = { 
    'n_estimators': [1, 5, 10, 25, 50, 75],
    'algorithm': ['SAMME', 'SAMME.R'],
    'learning_rate': [0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
}

CV_rfc = gs.GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3, scoring='log_loss')
CV_rfc.fit(X_train, y_train)
print CV_rfc.best_params_

#{'n_estimators': 5, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}


#http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
for params, mean_score, scores in CV_rfc.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))




{'n_estimators': 5, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}
-3.635 (+/-0.000) for {'n_estimators': 1, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 5, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 10, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 25, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 50, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 75, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 1, 'learning_rate': 0.1, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 5, 'learning_rate': 0.1, 'algorithm': 'SAMME'}
-3.635 (+/-0.000) for {'n_estimators': 10, 'learning_rate': 0.1, 'algorithm': 'SAMME'}
-3.636 (+/-0.000) for {'n_estimators': 25, 'learning_rate': 0.1, 'algorithm': 'SAMME'}
-3.636 (+/-0.000) for {'n_estimators': 50, 'learning_rate': 0.

In [5]:
clf = AdaBoostClassifier(n_estimators=5,learning_rate=0.1, algorithm='SAMME.R')
clf.fit(X_train, y_train) 
y_pred=clf.predict_proba(X_test)
y_true=y_test
y_true=y_true.values
classifier_results=log_loss(y_true, y_pred)
print classifier_results
              
#2.49779559817

2.49779559817


# Extra Trees

# This won't run because of an error in numpy
#https://github.com/pydata/pandas/issues/12600

rfc = ExtraTreesClassifier() 

param_grid = { 
    'n_estimators': [1, 5, 10, 25],
    'criterion': ['gini', 'entropy'],
    'max_features': ['auto','sqrt','log2',None],
    'bootstrap': [True,False],
    'oob_score': [False],
    'class_weight': ['balanced','balanced_subsample',None]
}

CV_rfc = gs.GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, scoring='log_loss', n_jobs=-1)
CV_rfc.fit(X_train, y_train)
print CV_rfc.best_params_





#clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=1, random_state=0)
#clf.fit(X_train, y_train) 
#y_pred=clf.predict_proba(X_test)
#y_true=y_test
#y_true=y_true.values
#classifier_results=log_loss(y_true, y_pred)
#print classifier_results
    

In [6]:
# Decision Trees
rfc = DecisionTreeClassifier(random_state=345) 

param_grid = { 
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_features': ['auto','sqrt','log2',None],
    'min_samples_split': [2,3,4,5],
    'min_samples_leaf': [1,2,3],
    'max_leaf_nodes': [25,30,35],
    'class_weight': ['balanced',None]
}

CV_rfc = gs.GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3, scoring='log_loss', n_jobs=1)
CV_rfc.fit(X_train, y_train)
print CV_rfc.best_params_

#{'splitter': 'best', 'max_leaf_nodes': 35, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': None, 'class_weight': None}


#http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
for params, mean_score, scores in CV_rfc.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))





{'splitter': 'best', 'max_leaf_nodes': 35, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': None, 'class_weight': None}
-2.515 (+/-0.075) for {'splitter': 'best', 'max_leaf_nodes': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 'auto', 'class_weight': 'balanced'}
-3.211 (+/-0.353) for {'splitter': 'random', 'max_leaf_nodes': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 'auto', 'class_weight': 'balanced'}
-2.515 (+/-0.075) for {'splitter': 'best', 'max_leaf_nodes': 25, 'min_samples_leaf': 1, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 'auto', 'class_weight': 'balanced'}
-3.211 (+/-0.353) for {'splitter': 'random', 'max_leaf_nodes': 25, 'min_samples_leaf': 1, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 'auto', 'class_weight': 'balanced'}
-2.515 (+/-0.075) for {'splitter': 'best', 'max_leaf_nodes': 25, 'min_samples_leaf': 1, 'min_samples_

In [7]:
clf = DecisionTreeClassifier(splitter='best', max_leaf_nodes=35, min_samples_leaf=1, min_samples_split=2, criterion='gini', 
                             max_features=None, class_weight=None)
clf.fit(X_train, y_train) 
y_pred=clf.predict_proba(X_test)
y_true=y_test
y_true=y_true.values
classifier_results=log_loss(y_true, y_pred)
print classifier_results

#1.83084523856

1.83084523856
